In [ ]:
!pip install geopandas


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
import math
from collections import namedtuple

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, TimestampedGeoJson

import datetime
import os

import json
from folium.features import DivIcon

geo_path = 'TL_SCCO_CTPRVN.json'
kr_geo_data = json.load(open(geo_path, encoding='utf-8'))
df = pd.read_csv('TimeProvince.csv')

In [ ]:
df

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0


In [ ]:
geo = pd.read_csv('geo_kr.csv')
geo

,city,latitude,longitude
0,Busan,35.179444,129.075556
1,Chungcheongbuk-do,36.821667,127.656944
2,Chungcheongnam-do,36.659411,126.673257
3,Daegu,35.871403,128.601751
4,Daejeon,36.350833,127.385000
5,Gangwon-do,37.750000,128.250000
6,Gwangju,35.159444,126.852500
7,Gyeonggi-do,37.500000,127.250000
8,Gyeongsangbuk-do,36.250000,128.750000
9,Gyeongsangnam-do,35.250000,128.250000


In [ ]:
df = df.rename({'province':'city'}, axis='columns')
df

,date,time,city,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0
...,...,...,...,...,...,...
2766,2020-06-30,0,Jeollabuk-do,27,21,0
2767,2020-06-30,0,Jeollanam-do,24,19,0
2768,2020-06-30,0,Gyeongsangbuk-do,1389,1328,54
2769,2020-06-30,0,Gyeongsangnam-do,134,128,0


In [ ]:
df_con = pd.merge(df, geo, on='city')
df_con

,date,time,city,confirmed,released,deceased,latitude,longitude
0,2020-01-20,16,Seoul,0,0,0,37.566671,126.978423
1,2020-01-21,16,Seoul,0,0,0,37.566671,126.978423
2,2020-01-22,16,Seoul,0,0,0,37.566671,126.978423
3,2020-01-23,16,Seoul,0,0,0,37.566671,126.978423
4,2020-01-24,16,Seoul,0,0,0,37.566671,126.978423
...,...,...,...,...,...,...,...,...
2766,2020-06-26,0,Jeju-do,19,16,0,33.366667,126.533333
2767,2020-06-27,0,Jeju-do,19,16,0,33.366667,126.533333
2768,2020-06-28,0,Jeju-do,19,16,0,33.366667,126.533333
2769,2020-06-29,0,Jeju-do,19,16,0,33.366667,126.533333


In [ ]:
df_con.to_csv(r'df_geo.csv', header=True, index=False)

In [ ]:
df_con = pd.read_csv(r'df_geo.csv')
df_on = pd.read_csv(r'df_geo.csv')

In [ ]:
def prepare_df_date(df):
    df['date'] = pd.to_datetime(df['date'])
    df['confirmed'] = df['confirmed'].fillna(0)
    return df

df_con = prepare_df_date(df_con)
df_on = prepare_df_date(df_on)

In [ ]:
df_on.dtypes

date         datetime64[ns]
time                  int64
city                 object
confirmed             int64
released              int64
deceased              int64
latitude            float64
longitude           float64
dtype: object

In [ ]:
route = pd.read_csv("PatientRoute.csv")

route_data = np.array([route.latitude, 
                 route.longitude]).T

In [ ]:
def create_geojson_features(df_con, df_on,
radius_max=1000,radius_min=20,
fill_color_confirmed='#FC766AFF',
fill_color_released='#0A5E2AFF',
fill_color_deceased='#E80018',
weight=1,
fill_opacity=0.5
):
    print('> Creating GeoJSON features...')

    features = []
    feature = []

    for _, row in df_con.iterrows():
        radius = np.sqrt(row['confirmed'])
        if radius != 0:
            if radius < radius_min:
                radius = radius_min
            if radius > radius_max:
                radius = radius_max

            popup = str(row['city']) + str(row['confirmed'])

            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'Point',
                    'coordinates': [row['longitude'], row['latitude']]
                },
                'properties': {
                    'time': row['date'].__str__(),
                    'style': {'color': fill_color_confirmed},
                    'icon': 'circle',
                    'iconstyle': {
                        'fillColor': fill_color_confirmed,
                        'fillOpacity': fill_opacity,
                        'stroke': 'true',
                        'radius': radius*3,
                        'weight': weight,
                        'popup': popup
                    }
                }
            }
            features.append(feature)

        for _, row in df_on.iterrows():
            radius = np.sqrt(row['confirmed'])
            if radius != 0:
                if radius < radius_min:
                    radius = radius_min
                if radius > radius_max:
                    radius = radius_max
            
            popups = str(row['city']) + str(row['confirmed'])
            size=radius*3, radius*3
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'Point',
                    'coordinates': [row['longitude'], row['latitude']]
                },
                'properties': {
                    'time': row['date'].__str__(),
                    'style': {'color': fill_color_confirmed},
                    'icon': 'marker',
                    'iconstyle': {'iconUrl': 'https://cdn.iconscout.com/icon/premium/png-512-thumb/coronavirus-4-613136.png',
                                  'iconSize': [radius*3, radius*3],
                                  'fillOpacity': 0.1,
                                  'popup': popups
                    }
                }
            }
            features.append(feature)

        print('> finishing GeoJSON features...')
        return features

In [ ]:
def make_map(features, caption):
    print('>Making map...')
    coords=[36.623537, 127.886186]
    m = folium.plugins.DualMap(location=coords,
                            control_scale=True,
                            zoom_start=7,
                            tiles='cartodbpositron',
                            detect_retina=True
                            )
    
    folium.TileLayer("OpenStreetMap").add_to(m.m1)
    folium.TileLayer("OpenStreetMap").add_to(m.m2)
    
    HeatMap(route_data).add_to(m.m1)

    folium.Choropleth(
        geo_data=kr_geo_data,
        name='choropleth',
        key_on='features.properties.name',
        fill_color='yellow',
        fill_opacity=0.15,
        line_opacity=0.7
        ).add_to(m.m2)
    
    TimestampedGeoJson(
        {'type': 'FeaturesCollection',
        'features': features},
        period='P1D',
        duration='P1D',
        add_last_point=True,
        auto_play=False,
        loop=False,
        max_speed=20,
        loop_button=True,
        date_options='MM/DD/YYYY',
        time_slider_drag_update=True,
        transition_time=500
    ).add_to(m.m2)
    folium.LayerControl(collapsed=False).add_to(m)
    
    m.caption = caption
    print('> Done.')
    m.save('Corona_dualmap.html')

    return m

features = create_geojson_features(df_con, df_on, fill_opacity=0.3, weight=1)
make_map(features, caption='Coronavirus propagation in Korea, 2020.')

> Creating GeoJSON features...
> finishing GeoJSON features...
>Making map...
> Done.
